# Radar data format:

Radars from the Australian Bureau of Meteorology use the ODIM conventions (EU/Canada WMO standards) and the HDF file format to store radar data (odimh5). This is supported by the Python library Py-ART and several other libraries.

In this notebook we will read a data file from the Bureau of Meteorology using Py-ART. We will plot the data and transform it's radar coordinates (spherical) to Cartesian space (x, y, z). In a later notebook, we will show how to apply this transformation to the data themselves (and not just for plotting), a process called gridding (interpolation + change of coordinates) to represent the data in x, y, z coordinates.

We will use data from a hailstorm event sample by the Melbourne radar on the 19 Jan 2020. Melbourne's Laverton radar is a dual-polarisation radar. While dual-polarisation radars are not the subject of this workshop, we think you should know a couple of things about them: 

### What dual-polarisation radars do?

- Theoretically, improve the accuracy of precipitation estimates, leading to better flash flood detection,
- Ability to discern between heavy rain, hail, snow, and sleet,
- Improved discrimination of non-meteorological echoes (e.g., ground clutter, chaff, anomalous propagation, birds, and tornado debris),
- Identification of the melting layer (e.g., bright band)

### What dual-polarisation radars don't do yet?

- Will not improve tornado lead times,
- Will not provide exact precipitation type at ground level,
- Will not make coffee.

### What is different from single-polarisation radars?

Additional information from dual-pol can determine (as a function of the sampled volume): 
- particle shape
- sample heterogeneity
- water content

### Let's look at some data

Because radars produce a volume every 5 minutes (sometimes 6 min, sometimes 10 min for older radar), it can be a challenge to know when there's an interesting case study, or something worth to look at, when dealing with historical data. Radar volumes for a day are in the several gigabytes and with 240 files to explore, it becomes quickly a tedious task. To deal with that we often create quicklooks, simple plots that give us a basic idea on what to expect.


![](img/opol_quicklooks_2019-12-25.png)

In the figure above we can see the daily maximum reflectivity over the radar domain, and the frequency of occurence of echoes above a 40 dBZ threshold, that can be used as first approximation to identify convective rain. 

The first time series show the fraction of the radar domain occupied by stratiform/convective rain (here using a simple threshold). The other timeseries are vertical profiles (QVP) of several quantities that the radar measures. The reflectivity, the differential reflectivity (an indicator of the shape), the differential phase $(\phi_{dp}$, an indicator of the medium that the waves go through), and the cross correlation ratio $(\rho_{hv}$, an indicator if we are measuring hydrometeors). $\rho_{hv}=1$ in rain, $\rho_{hv}<0.6$ corresponds to debris, clear-air, non-meteorological echoes.

The OceanPOL data contains:
- Air echo classification
- Corrected differential phase
- Corrected differential reflectivity
- Corrected reflectivity
- Corrected specific differential phase
- Corrected velocity
- Cross correlation ratio
- Differential phase
- Differential reflectivity
- Normalized coherent power
- Radar echo classification
- Radar estimated rain rate
- Signal to noise ratio
- Spectrum width
- Temperature (ECMWF reanalysis mapped to each radar gate)
- Total power (raw reflectivity)
- Velocity


In [ ]:
%matplotlib inline

import pyart
import cftime
import cartopy.crs as ccrs
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#define radar file
radar_file = 'data/OceanPOL-20191225-123000.nc'

In [ ]:
#read radar file into memory using pyart
radar = pyart.io.read(radar_file)

In [ ]:
#inspect radar fields
radar.fields.keys()

## How to deal with Spherical coordinates?

Radars are fixed instrument, generally put on top of a tower. They scan the atmosphere by spinning. They usually spin 360°, then change elevation (tilt higher) then spins 360° again, several time, up until we have a 3D representation of the atmosphere. Thus, by spinning and changing elevation, radar data are in **spherical coordinates**. However, not the *mathematical* definition of the spherical coordinates, it's the nautical definition. The azimuth starts at North and goes clock-wise (while mathematically it starts on the x-axis and goes counter-clockwise). Also, for radars coordinates the elevation goes upward, from ground to zenith (while in math the elevation goes from the z-axis, up, to the xy-plane, down).

In short, 0° azimuth is North, 90° is East, 180° is South and 270° is West, increasing clockwise. For the elevation, 0° is the plane parallel to the ground and 90° is the zenith.
![coords](img/coords.png)

- Let's read one radar sweep
- Transform the bearing angles into mathematical angles
- Transform the spherical coordinates to Cartesian.

In [ ]:
#get index values of 2nd sweep (Python index of 1)
sweep = radar.get_slice(2)

#read range and azimuth dimensions
r = radar.range['data']  # to km
radar_azi = radar.azimuth['data'][sweep]
#read reflectivity dataset
reflectivity = radar.fields['total_power']['data'][sweep]
# Read the date.
date = cftime.num2pydate(radar.time['data'][0], radar.time["units"])

In [ ]:
#plot PPI in range/azimuth Cartesian space
fig = plt.figure(figsize=(10, 8))
plt.pcolormesh(r, radar_azi, reflectivity, cmap='pyart_HomeyerRainbow', vmin=-15, vmax=65)
plt.xlabel('Range from radar (m)')
plt.ylabel(r'Azimuth $(^\circ)$ - 0 is North')
plt.colorbar(label='Reflectivity (dBZ)')
plt.title(f'OceanPOL radar - {date.isoformat()}')
plt.xlim(0, 150e3)
plt.show()

In [ ]:
# From angle bearing to mathematical angle.
azimuth = np.pi / 180 * ((450 - radar_azi) % 360)  
[R, A] = np.meshgrid(r, azimuth)

In [ ]:
# From spherical to Cartesian
x = R * np.cos(A)
y = R * np.sin(A)

In [ ]:
#plot PPI in range/azimuth Polar space
fig = plt.figure(figsize=(10, 8))
plt.pcolormesh(x, y, reflectivity, cmap='pyart_HomeyerRainbow', vmin=-15, vmax=65)
plt.xlabel('x-coordinates from radar (m)')
plt.xlabel('y-coordinates from radar (m)')
plt.colorbar(label='Reflectivity (dBZ)')
plt.title(f'OceanPOL radar - {date.isoformat()}')
plt.xlim(-150e3, 150e3)
plt.ylim(-150e3, 150e3)
plt.show()

### Let's plot the data onto a map

In [ ]:
#define mapping projection
projection = ccrs.AzimuthalEquidistant(central_longitude=radar.longitude['data'][0],
                                       central_latitude=radar.latitude['data'][0])

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax1 = plt.subplot(1, 1, 1, projection=projection)
ax1.coastlines('10m')
im = ax1.pcolormesh(x, y, reflectivity, cmap='pyart_HomeyerRainbow', vmin=-15, vmax=65)
ax1.set_xlim(-150e3, 150e3)
ax1.set_ylim(-150e3, 150e3)
plt.colorbar(im, label='Reflectivity (dBZ)')
plt.title(f'OceanPOL radar - {date.isoformat()}')
plt.show()

# Py-ART makes things easier

In [ ]:
gr = pyart.graph.RadarDisplay(radar)

In [ ]:
gr.plot_ppi("total_power", sweep=2)
plt.show()